In [ ]:
import os
os.listdir()

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from dateutil import parser

In [ ]:
#Load in appointment data

appointments = pd.read_csv('AHSN_Final_Taylor_Appointments_Total.csv', sep ='|')
appointments

In [ ]:
#Keep relevant columns

appointments = appointments[['PatientId', 'DtStart', 'AppointmentStatus']]
appointments = appointments.sort_values(by=['PatientId', 'DtStart'])

In [ ]:
#Print number of appointment records

len(appointments)

In [ ]:
#Keep only records wher appointment status is 'finished'

finished_appointments = appointments.ix[appointments['AppointmentStatus'] == 'Finished']
finished_appointments.head(100)

In [ ]:
#Load in date of first diagnosis

first_diagnosis = pd.read_csv('first_diagnosis.csv')
first_diagnosis = first_diagnosis[['PatientId', '1st_SMI_Diagnosis']]
first_diagnosis

In [ ]:
#Pulling in data about date of death

death = pd.read_csv('all_data.csv')
death = death[['PatientId','DateOfDeath']]
death = death.drop_duplicates()
death

In [ ]:
#Merge first diagnosis with death date and appointment history (df_app)

df_appointments = first_diagnosis.merge(death, on = 'PatientId', how = 'left').merge(finished_appointments, on='PatientId', how = 'left')
df_appointments = df_appointments.sort_values(by=['PatientId', 'DtStart'])
df_appointments

In [ ]:
#Print number of unique patients in 'df_appointments'

len(set(df_appointments['PatientId']))

In [ ]:
#Drop NaNs in 'appointment status' column

df_appointments_NoNaN = df_appointments.dropna(subset=["AppointmentStatus"])
df_appointments_NoNaN


In [ ]:
#Convert dates to datetime objects

df_appointments_NoNaN['DtStart'] = pd.to_datetime(df_appointments_NoNaN['DtStart'])
df_appointments_NoNaN['1st_SMI_Diagnosis'] = pd.to_datetime(df_appointments_NoNaN['1st_SMI_Diagnosis'])
df_appointments_NoNaN['DateOfDeath'] = pd.to_datetime(df_appointments_NoNaN['DateOfDeath'])
df_appointments_NoNaN

In [ ]:
#remove appointments before diagnosis

df_working = df_appointments_NoNaN.ix[df_appointments['DtStart'] >= df_appointments['1st_SMI_Diagnosis']]


In [ ]:
#remove appointments before 2016

df_working = df_working.ix[df_appointments['DtStart'] <= '2016-01-01']

In [ ]:
#Print number of patients still alive

sum(df_working.DateOfDeath.isnull())

In [ ]:
#Create new column for year of diagnosis
df_working['YearOfDiagnosis'] = df_working['1st_SMI_Diagnosis'].dt.year

#Create new column for year after diagnosis (i.e. first full year for which appointment history available after diagnosis)
df_working['YearAfterDiagnosis'] = df_working['YearOfDiagnosis'] + 1
df_working

In [ ]:
#Create new dataframe including only patient if still alive (ie. Date of Death = NaT)

df_working_1 = df_working.ix[df_working['DateOfDeath'].isnull()]
df_working_1


In [ ]:
#Replace 'Trues' in Null column with '2015'. I.e. last full year for which patient records are available

df_working_1['IsNull'] = 2015
df_working_1

In [ ]:
#Create new dataframe including only patient that have died (ie. Date of Death = Datetime object)

df_working_2 = df_working.ix[df_working['DateOfDeath'].notnull()]
df_working_2

In [ ]:
#Replace 'Falses' in Null column with 'Year of death'. Subtract 1, so that only data for last full year included.

df_working_2['IsNull'] = (df_working_2['DateOfDeath'].dt.year-1)
df_working_2

In [ ]:
#Merge dataframes of 'living' and 'dead' patients
df_working_merge = df_working_1.merge(df_working_2, how = 'outer')

#Create new column (Enddate)
df_working_merge['EndDate'] = df_working_merge['IsNull']
df_working_merge

In [ ]:
#Make sure patients only included where at least 1 full year of appointment history is available

df_working_merge['Difference'] = (df_working_merge['EndDate'] - df_working_merge['YearAfterDiagnosis']) + 1
df_current = df_working_merge.ix[df_working_merge['Difference'] >= 1]
df_current



In [ ]:
#Remove appointments outside of time window

#Create new column containing appointment years 

df_current['Appointment_Year'] = df_current['DtStart'].dt.year
df_current

In [ ]:
df_current_mod = df_current.ix[df_current['Appointment_Year'] >= df_current['YearAfterDiagnosis']]
df_current_mod

In [ ]:
#Tidy things up a bit

df_trimmed = df_current_mod[['PatientId', 'Appointment_Year', 'YearAfterDiagnosis', 'EndDate', 'Difference']]
df_trimmed

In [ ]:
#Print number of unique patients that have appointment records

print('Unique Patients: ', len(set(df_trimmed['PatientId'])))

In [ ]:
#Print number of patient records available for each patient (i.e. total number of appointments)

summary = df_trimmed.groupby('PatientId').count()
summary 


In [ ]:
#Keep only 1 column

totals = summary['Appointment_Year']

In [ ]:
#Convert table to dataframe
totals = pd.DataFrame(totals)
totals['PatientId'] = totals.index
totals['Total Appointments'] = totals['Appointment_Year'] 
totals = totals[['PatientId', 'Total Appointments']]
totals

In [ ]:
#Merge df_trimmed and totals
df_nearly = df_trimmed.merge(totals, on = 'PatientId', how = 'left')

#Add a column of 'Appointments per year' (Total appointments/Difference)
df_nearly['Appointments per year'] = df_nearly['Total Appointments']/df_nearly['Difference']
df_nearly

In [ ]:
#Select the desired columns
df_close = df_nearly[['PatientId', 'Appointment_Year', 'YearAfterDiagnosis', 'EndDate', 'Difference', 'Total Appointments', 'Appointments per year']]

#Rename columns
df_close.rename(columns={'YearAfterDiagnosis': '1st_year', 'EndDate': 'Last_year', 'Difference': 'Number_of_years'}, inplace=True)
df_close

# Getting average number of appointments for every year after diagnosis

In [ ]:
#Adding a column showing 'Number of years after diagnosis
df_close['Number of years after diagnosis'] = (df_close['Appointment_Year'] - df_close['1st_year']) +1
df_close

In [ ]:
#Group data primarily by 'Patient ID', then by 'Number of years after diagnosis'

grouped_id = df_close.groupby(['PatientId','Number of years after diagnosis']).count()
grouped_id

In [ ]:
#Keep only 1 column

Annual_appointments_record = grouped_id['Appointments per year']
Annual_appointments_record

In [ ]:
#Create final table summarising appointment data

df_done = df_close.drop_duplicates()
df_done = df_done.round(2)
df_done = df_done.drop("Number of years after diagnosis",1)
df_done = df_done[['PatientId', '1st_year', 'Last_year', 'Number_of_years', 'Total Appointments', 'Appointments per year']]
df_done = df_done.drop_duplicates()
df_done

In [ ]:
#Export df_done as Excel File ('Part4_Appointment_History.csv')

df_done.to_csv('Part4_Appointment_History.csv', sep=',')

In [ ]:
#Create copy of Annual_appointments_record

AAR = Annual_appointments_record.copy()
AAR

In [ ]:
#Reset index of AAR

AAR = AAR.reset_index()

In [ ]:
#Remove duplicate rows

AAR = AAR.drop_duplicates()
AAR

In [ ]:
#Pivot AAR

AAR_pivoted = AAR.pivot(index='PatientId', columns='Number of years after diagnosis').fillna('-')
AAR_pivoted

In [ ]:
#Export AAR_pivoted as Excel File ('Part4_Annual_Appointment_record.csv')

AAR_pivoted.to_csv('Part4_Annual_Appointment_record.csv', sep=',')